<img src=https://desafiolatam.com/assets/home/logo-academia-bla-790873cdf66b0e681dfbe640ace8a602f5330bec301c409744c358330e823ae3.png width="300">

# <div style="text-align: center"> Documento Técnico Proyecto AutoPGS</div>                                          
# <div style="text-align: center"> Automobile Purchase Guide System</div>

<p style="text-align: justify;">Nuestra empresa es Data Machine Consulting Group se encarga de realizar consultorías por medio de análisis de datos para una variedad de clientes buscando cumplir con sus objetivos.
Realizaremos un reporte completo del proyecto Automobile Purchase Guide System, el cual nace a solicitud de nuestro cliente Automotora Anaconda y consiste en generar una herramienta predictiva que ayude a su equipo de compras a encontrar las mejores oportunidades para la obtención y posterior venta de automóviles usados, mediante una serie de pasos que se detallaran a continuación.

Abordaremos temas relevantes que van desde la preparación del ambiente de trabajo, recolección e importación de datos, como también la limpieza, análisis, recodificación y segmentación de los mismos, además del modelamiento y análisis comparativo de los resultados.

Por último se realizará la implementación de una aplicación web que permita en base a una cierta cantidad de atributos propios de cada vehículo estimar su valor y realizar una recomendación considerando las políticas propias del negocio y los precios de mercado, que finalmente se traduzcan para nuestros clientes en un aumento de las ventas y una mayor utilidad por cada vehículo  comercializado, como también una mayor rotación de estos y variedad de las unidades disponibles.</p>

## Preparación del ambiente de trabajo:

<p style="text-align: justify;">Antes de comenzar la implementación de este proyecto se requiere importar una serie de librerías que permitan realizar la ingesta, análisis y  modelación de los datos, dentro de los parámetros generales del notebook definiremos la ruta donde se alojaran los archivos csv a utilizar, bases de datos, como también el estilo y tamaño de los plot, gráficos para analizar, y se dejará definido el número de semilla aleatoria que se utilizaran para los distintos modelos.</p>

In [1]:
# Parámetros generales del notebook
ruta_bases = 'bases/'

In [2]:
# Importación de librerías
import json
import glob
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import funciones as fn
import seaborn as sns
import scipy.stats as stats
import pylab 
import warnings
warnings.filterwarnings('ignore')
from ml_classes import PrepML, MLModel
from matplotlib.pyplot import rcParams
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge,SGDRegressor
from lib.get_nhtsa_json import get_nhtsa_json
from pandas.api.types import CategoricalDtype

KeyboardInterrupt: 

In [ ]:
# Parámetros generales para plots
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 15, 8
plt.style.use('ggplot')
# Semilla pseudo-aleatoria
rd_seed = 1234

# Extracción de los datos

<p style="text-align: justify;">Para realizar este proyecto dispondremos de 3 dataset en formato cvs entregados por nuestro cliente, `true_cars_train.csv` y `true_cars_test.csv`, cada uno de ellos contiene 639.145 y 212.977 registros respectivamente, y un tercer dataset bajo el nombre de `true_car_listings.csv`, se reúne la totalidad datos de los dos primeros archivos. 

Cada una de estos archivos cuentan con ocho atributos propios de cada vehículo dentro de los que tenemos (Precio, Año, Millas, Ciudad, Estado, Vin, Marca y Modelo), dada la poca cantidad de atributos con la que contamos, se hace necesario considerar la búsqueda y extracción de nuevos datos, adicionales a los que ya tenemos para cada vehículo, esto con el propósito de mejorar la exactitud del modelo y lograr una mejor estimación de nuestro vector objetivo `Precio`, para ello utilizaremos el número único identificador de vehículos, o más conocido como `Vin`, el cual es una secuencia de dígitos que identifica los vehículos motorizados de cualquier tipo.

El proceso de extracción de estos datos adicionales se realizará utilizando el número único `Vin` que ya tenemos definido para cada vehículo, proceso el cual se logró gracias la creación y ejecución de la función `get_features.py`, mediante la conexión a la API en el sitio web: https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/, cabe destacar que esta pagina pertenece a la NHTSA, siendo esta la Administración Nacional de Seguridad del Tráfico en las Carreteras, agencia dependiente del gobierno de los Estados Unidos que reúne la información requerida.

A modo de ejemplificar el proceso de extracción de los datos en el siguiente script se presenta una demostración del proceso realizado por la función `get_features.py`, la cual se resume en los siguientes pasos:

* 1.- Extraer todos los `Vin` de la base completa `true_car_listings.csv`.
* 2.- Requerir a través de la Api 'chunks' de 50 registros por cada petición.
* 3.- Guardar en la memoria el json en formato texto, agregando 50 registros por cada iteración.
* 4.- Una vez completada las iteraciones guardar en formato json todos los registros requeridos.</p>

```python
# Requerimos todos los 'Vin'    
all_vins = pd.read_csv(f'{ruta_bases}true_car_listings.csv')['Vin'].to_list()
# Parámetros de muestra
start = 8
end = 8

json_text = '['
for i in range(start, end+1):
    # Generar requerimiento con 50 registros Vin
    vin_list = all_vins[50 * (i - 1):50 * i]
    json_text += get_nhtsa_json(vin_list, i)

# Cerrar lista de Json
json_text = json_text[:-2] + ']'
# Exportar resultados a archivo json
with open(f'api_test/data_{start}_{end}.json', 'w') as json_file:
    json_file.write(json_text)
```

Paralelamente, en base a una muestra de la base total, se definió el primer filtro de variables requeridas a través la API: que tengan menos del 10% de datos perdidos, las cuales se presentan a continaución:

```python
cols = ['AirBagLocFront', 'BodyClass', 'BusFloorConfigType', 'BusType',
       'CustomMotorcycleType', 'DisplacementCC', 'DisplacementCI',
       'DisplacementL', 'Doors', 'EngineCylinders', 'EngineHP', 'EngineKW',
       'ErrorCode', 'ErrorText', 'FuelTypePrimary', 'Make', 'Manufacturer',
       'ManufacturerId', 'Model', 'ModelYear', 'MotorcycleChassisType',
       'MotorcycleSuspensionType', 'PlantCity', 'PlantCountry', 'TPMS',
       'TrailerBodyType', 'TrailerType', 'VIN', 'VehicleType']
```

Con estas columnas seleccionadas, se procede a importar los archivos json (varios en el proceso original) para luego mapearlos para retraer solo aquellas columnas, creando un DataFrame con ellas y luego exportarlas en un csv.

```python
# Importación de archivo json
filenames = glob.glob('api_test/*.json')
json_list = []

for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        # Mapeamos considerando solo las columnas seleccionadas
        data = list(map(fn.get_info, 
                        json.loads(file.read())
                       )
                   )
    json_list += data

# Creación y exortación de DataFrame con features extraídos
data_json = pd.DataFrame(data=json_list,
                         columns=cols)
data_json.to_csv('api_test/data_api.csv')
````

# 1. Ingesta y creación del Dataset 

Para comenzar con el análisis de los set de datos a utilizar, mediante la función `read_csv` de pandas leemos los archivos `true_cars_train.csv` y `true_cars_test.csv`, adicionalmente a cada uno de estos se les agrega una nueva columna llamada `sample`, la cual permitirá identificar el origen de cada uno de los registros de la base de datos, por último mediante la función `concat` de pandas realizaremos la unión de ambos archivos, pasando así a tener un solo gran archivo dataFrame con la totalidad de los registros, el cual queda bajo el nombre de `df_data`.

## 1.1 Bases Originales

In [ ]:
# Importación de las bases para muestras de entrenamiento y prueba
df_train = pd.read_csv(f'{ruta_bases}true_cars_train.csv',
                       delimiter=";")
df_test = pd.read_csv(f'{ruta_bases}true_cars_test.csv',
                      delimiter=";")
# Dimensiones de las bases
print(f'Base Train: {df_train.shape}\nBase Test: {df_test.shape}')

In [ ]:
# Información general de muestra de entrenamiento
df_train.info()

In [ ]:
# Creación de atributo 'sample'
df_train['sample'] = 'train'
df_test['sample'] = 'test'

In [ ]:
# Unión de ambas bases
df_data = pd.concat([df_train, df_test])
print(f'Base Data: {df_data.shape}')

__Observaciones__:

Mediante `info()`, podemos observar que los atributos del DataFrame no contienen datos nulos, tres de ellos son de tipo númericos `int64` y los restantes cinco son de tipo `objeto`.

## 1.2 Base API

El archivo obtenido gracias a la función `get_features.py` queda alojado en la carpeta `ruta_bases` bajo el nombre de `api_features.csv`, mediante `read_csv` de pandas realizamos la ingesta de los datos asigandolos al objeto `df_api`, el cual podemos observar que contiene 846.562 registros y 29 columnas que nos proporcionan nuevas caracteristicas para cada cada uno de los `Vin` de los vehiculos consultados.

In [ ]:
# Importación de la base extraída por el requerimiento a la api
df_api = pd.read_csv(f'{ruta_bases}api_features.csv').drop(columns='Unnamed: 0')
print(f'Base API: {df_api.shape}')

In [ ]:
df_api.info()

In [ ]:
# Identificar columnas con solo valores "Not Applicable" 
notapp_series = df_api\
                    .isin(['Not Applicable'])\
                    .sum()
cols2drop = list(notapp_series[notapp_series > 1].index)

In [ ]:
# Borrar aquellas columnas 
df_api = df_api.drop(columns=cols2drop)
print(f'Base API: {df_api.shape}')

In [ ]:
# Renombrar columnas
df_api=df_api.rename(columns={"AirBagLocFront":"d_AirBagLocFront",
                        "BodyClass":"d_BodyClass",
                        "DisplacementCC":"d_DisplacementCC",
                        "DisplacementCI":"d_DisplacementCI",
                        "DisplacementL":"d_DisplacementL",
                        "Doors":"d_Doors",
                        "EngineCylinders":"d_EngineCylinders",
                        "EngineHP":"d_EngineHP",
                        "EngineKW":"d_EngineKW",
                        "ErrorCode":"d_ErrorCode",
                        "ErrorText":"d_ErrorText",
                        "FuelTypePrimary":"d_FuelTypePrimary",
                        "Make":"d_Make",
                        "Manufacturer":"d_Manufacturer",
                        "ManufacturerId":"d_ManufacturerId",
                        "Model":"d_Model",
                        "ModelYear":"d_ModelYear",
                        "PlantCity":"d_PlantCity",
                        "PlantCountry":"d_PlantCountry",
                        "TPMS":"d_TPMS",
                        "VIN":"Vin",
                        "VehicleType":"d_VehicleType"})

__Observaciones:__

Mediante `info()`, podemos observar que los atributos del DataFrame `df_api`, son de tipo objeto y float, se identifica además que existen datos tipificados como `Not Applicable`, los cuales se proceden a eliminar debido a que no generan un mayor aporte al desarrollo del proyecto, finalmente se renombran las columnas con el propósito de lograr una mejor identificación de cada atributo.

## 1.3 Unión de Bases

Ya con los archivos asignados a los Dataframes `df_data` y `df_api`, se procede mediante el método `merge` a su unión, en donde se utiliza el numero `Vin` como llave para generar la intercepción de los datos, quedando así una única gran base de 846.644 registros y 30 columnas, la cual se asigna bajo el nombre de `df`.

In [ ]:
# Unión de bases
df = pd.merge(left=df_data, 
              right=df_api, 
              how='inner',
              on='Vin')
# Dimensiones de la base
print(f'Dataset: {df.shape}')

In [ ]:
# Información general del dataset
df.info()

__Observaciones:__

Mediante la función `info()` aplicado al `df` final (que contiene la unión de ambos archivos), podemos observar que los atributos del DataFrame son de tipo `int`, `objet` y `float`.

## 1.4 Limpieza del Dataset

Dentro de los nuevos atributos obtenidos se destaca la columna `ErrorText`, la cual entrega información relevante respecto a la extracción de los datos e indica en ella si los atributos asociados al numero `Vin` se extrajeron de manera exitosa, es por esto que se toma la decisión de mantener solo aquellos datos que se obtuvieron de manera correcta pasando de tener en un inicio 846.644 a 845.778 registros.

### 1.4.1 ErrorText

In [ ]:
# revisamos la columna ErrorText, que entrega información respecto errores en la extracción de los datos
df['d_ErrorText'].value_counts().sort_values()

In [ ]:
# Se mantienen solo los datos extraídos correctamente
df = df[(df['d_ErrorText'] == '0 - VIN decoded clean. Check Digit (9th position) is correct') |
       (df['d_ErrorText'] == '0 - VIN decoded clean. Check Digit (9th position) is correct; 14 - Unable to provide information for all the characters in the VIN.')]

In [ ]:
#se eliminan las columnas de error que ya utilizamos
df = df.drop(columns=["d_ErrorCode","d_ErrorText"])

In [ ]:
# Limpieza de atributo
df["d_EngineCylinders"] = df["d_EngineCylinders"].map(lambda x: float(str(x)\
                                                                   .replace('12, 8', '12')\
                                                                   .replace('8, 12', '8'))
                                                                   )

__Observaciones:__

Para asegurar la calidad de los datos solo se mantendrán aquellos registros que fueron decodificados de manera correcta, adicionalmente a esto, se recodifican los datos de la columna `d_EngineCylinders`, dejándolos todos de tipo `float` para evitar posteriores errores al momento de graficar esta variable.

### 1.4.2 Nulos, datos duplicados y columnas duplicadas

<p style="text-align: justify;">Una de las consideraciones que se debe tener presente al momento de trabajar con grandes volúmenes de datos son precisamente los datos nulos, duplicados y columnas duplicadas, ya que estos pueden afectan el desarrollo del proyecto, tanto en la parte de análisis de los datos como en el entrenamiento y modelación. Es por esto que se decide eliminar las columnas duplicadas, como también  aquellas que presenten sobre el 0.15 % de datos nulos, asegurando así la calidad e integridad de los mismos.</p>

In [ ]:
#Datos nulos
#Mediante "isnull()", sum() y shape() identificamos los atributos con datos nulos y su procentaje relativo.
null = round(df.isnull().sum()/df.shape[0],2) 
null[null>0]

In [ ]:
# Identificar columnas con más de un 15% de datos perdidos y luego los eliminamos
null_series = df_api\
                .isnull()\
                .sum()\
                /df_api.shape[0] 
df = df.drop(columns=list(null_series[null_series > .15].index))

In [ ]:
#Datos duplicados
#Observamos la cantidad de registros duplicados en el df.
duplicate_rows_df = df[df.duplicated()]
duplicate_rows_df.shape

In [ ]:
# Eliminamos los registros duplicados
df=df.drop_duplicates()

In [ ]:
# Columnas duplicadas
# Chequemos si la columna Model, (data set de la academia) y la columna d_Model (data de la API) son iguales.
(df.Model==df.d_Model).value_counts("%")

In [ ]:
# vemos los primeros 5 registros de model y d_model.
df[['Model', 'd_Model']] .head(5)

In [ ]:
# chequemos si la columna Make, (data set de la academia) y la columna d_Make (data de la API) son iguales.
(df.Make==df.d_Make).value_counts("%")

In [ ]:
# Valores unicos columna make.
np.unique(df.Make)

In [ ]:
# Valores unicos columna d_make.
np.unique(df.d_Make)

In [ ]:
# Chequemos si la columna Year, (data set de la academia) y la columna d_ModelYear (data de la API) son iguales.
(df.Year == df.d_ModelYear).value_counts("%")

In [ ]:
# Eliminación columnas duplicadas provenientes de la API.
df = df.drop(columns=['d_Make',"d_Model","d_ModelYear", 'd_ManufacturerId',
                      'd_DisplacementCI', 'd_DisplacementL'])

In [ ]:
# Eliminación de variables no informativas
df = df.drop(columns=['Vin'])

__Observaciones:__

<p style="text-align: justify;">En general podemos observar que los porcentajes de valores nulos por atributo son bastante bajos, a excepción de `d_engineHP`, `d_EngineKW`, `d_TPMS` y `PlantCity`, los cuales superan el 0.15 % definido en un comienzo, bajo este escenario se procede a eliminarlos del dataset. 

Respecto a los registros y columnas duplicadas, se detectan 161 registros bajo esta condición, por lo que se procede a su exclusión, se decide también eliminar las columnas `d_Make`, `d_Model`, `d_modelYear`, `d_ManufacterID`, `d_DisplacenemetCI` y `d_DisplacementL` que se obtuvieron gracias a la extracción de datos desde la API ya que están duplicadas, y no entregan un mayor aporte al proyecto.

Dentro de los criterios que se utilizaron para definir la eliminación de atributos duplicados, tenemos  el caso  `d_Make` el cual al compararlo con el atributo original `Make`, podemos observar que  los nombres de las marcas no vienen normalizados, ademas de contener una menor cantidad de información, por otra parte al contrastar  `d_Model` con la columna original `Model`, se observa que esta contiene una menor cantidad de datos, criterios como estos se tomaron en consideraron al momento de la eliminación de atributos, quedándonos finalmente con un DataFrame que se conforma de 845.617 registros y 17 columnas.</p>

In [ ]:
# Inspección general después de limpieza
df.info()

# 2. Análisis exploratorio de datos

## 2.1 Distribución vector objetivo

<p style="text-align: justify;">Al observar la distribución del atributo precio (vector objetivo), podemos ver que este es muy variable, teniendo registros con valores muy bajos que parten en los  USD 1.500  como también otros que se escapan del promedio de forma destacada alcanzando hasta los USD 499.500. También observamos que la mayoría de los datos se encuentran entre los rangos 1.500 a 100.000 USD. 

Lo anterior se debe a que dentro de la BDD tenemos diferentes tipos de vehículos cada uno de estos con precios y características propias, por lo que a modo de segmentar estos atributos se decide renombrar las columnas según la tipologías asociadas al uso, tipo de vehículo, características técnicas, ubicación y lugar de fabricación, definiendo de este modos una mejor forma de analizar dichos atributos.</p>

- `USE (USE)`:Atributos relacioados al uso del auto:
    * __Year__, 
    * __Mileage__

- `SEGMENT (SGT)`:Atributos asociados al tipo de vehículo:
    * __Vehiculetype__, 
    * __Bodyclass__, 
    * __Make__, 
    * __Model__, 
    * __Manufacture__

- `FEATURES (FEAT)`:Atributos asociados las características técnicas del vehículo:
    * __Fuel type__, 
    * __Airbag__, 
    * __Displacement__,
    * __Doors__, 
    * __EngineCylindren__

- `LOCALIZATION (LOC)`:Atributos asociados a la ubicación del vehículo:
    * __State__,
    * __City__

- `FABRICATION(FAB)`:Atributo de ubicación de la fabricación:
    * __Plantcountry__

In [ ]:
## observamos la distribución del precio mediante un histograma y un gráfico de cajas y bigotes.
fn.distrbution_graph(df.Price)

In [ ]:
#Se renombran los atributos, según lo indicado anteriormente
df=df.rename(columns={"d_AirBagLocFront":"feat_AirBagLocFront",
                        "d_DisplacementCC":"feat_DisplacementCC",
                        "d_Doors":"feat_Doors",
                        "d_EngineCylinders":"feat_EngineCylinders",
                        "d_EngineHP":"feat_EngineHP",
                        "d_EngineKW":"feat_EngineKW",
                        "d_TPMS":"feat_TPMS",
                        "EngineCylinders":"feat_EngineCylinders",  
                        "d_FuelTypePrimary":"feat_FuelTypePrimary",
                        "d_BodyClass":"sgt_BodyClass",  
                        "d_Manufacturer":"sgt_Manufacturer",
                        "d_VehicleType":"sgt_VehicleType",  
                        "Model":"sgt_Model",
                        "Make":"sgt_Make",
                        "d_PlantCity":"fab_PlantCity",
                        "d_PlantCountry":"fab_PlantCountry",
                        "Year":"use_Year",
                        "Mileage":"use_Mileage",
                        "City":"loc_City",
                        "State":"loc_State"})

## 2.2 Distribución y relaciones de los atributos con el vector objetivo:
 
A continuación se realizaran una serie de gráficos para observar el comportamiento de los atributos asociados a las características técnicas del vehículo como son: `Fuel type`, `Airbag`, `Displacement`, `Doors` y `EngineCylindren`, además de realizar una matriz de correlación de los atributos de tipo numérico con el vector objetivo.

### 2.2.1 Atributos features (feat):

#### 2.2.1.1 Matriz de correlación entre atributos "feat" númericos u ordinales con el vector objetivo.

In [ ]:
# Generamos lista con columnas feat
col_names_feat = [col for col in df.columns if 'feat' in col]

In [ ]:
#generamos df con columnas feat más Price para hacer la matrix de correlación"
df_feat = df[col_names_feat]
df_feat["Price"] = df["Price"]

In [ ]:
# generamos matriz de correlación entre los atributos númericos u ordinales y nuestro vector objetivo precio.
f,ax = plt.subplots(figsize=(6, 4))
sns.heatmap(df_feat.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

### 2.2.1.2 Distribución y relación de los atributos categóricos "feat con el vector objetivo.

In [ ]:
#Variable "feat_airbag"
fn.count_box_plot('feat_AirBagLocFront', df)

In [ ]:
#Variable "feat_Doors"
fn.count_box_plot("feat_Doors", df)

In [ ]:
#Variable "feat_EngineCylinders"
fn.count_box_plot("feat_EngineCylinders",df, 200000)

In [ ]:
fn.count_box_plot("feat_FuelTypePrimary", df)

__Observaciones:__

- Los atributos `feat_Doors`, `feat_Airbag` y `feat_FuelType` están considerablemente desbalanceadas hacia una categoría, por lo que no se recomienda utilizar estas variables para el modelamiento de datos.
- `Displacement` presentan una correlación de 0.3 con el vector objetivo y se sugiere considerarla en el modelamiento. 
- Respecto a la variable `feat_EngineCylinders` presenta una correlación positiva con el precio de 0.4, por lo que también es un atributo candidato para ser utilizado en el proceso de entrenamiento.

### 2.2.2 Distribución atributos Use:

Se realizará una serie de gráficos para observar el comportamiento de los atributos asociados al `uso` del vehículo como lo son: `Year` y `Mileage`, además de generar una matriz de correlación de estos con el vector objetivo.

#### 2.2.2.1 Matriz de correlación entre atributos "Use" númericos u ordinales con el vector objetivo.

In [ ]:
# Generamos lista con columnas feat
col_names_use = [col for col in df.columns if 'use' in col]

In [ ]:
#generamos df con columnas use más Price para hacer la matrix de correlación"
df_use = df[col_names_use]
df_use["Price"] = df["Price"]

In [ ]:
# generamos matriz de correlación entre los atributos númericos u ordinales y nuestro vector objetivo precio.
f,ax = plt.subplots(figsize=(6, 4))
sns.heatmap(df_use.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

#### 2.2.2.2 Distribución y relación de los atributos categóricos "Use" con el vector objetivo.

In [ ]:
# Variable use_Mileage
fn.distrbution_graph(df.use_Mileage)

In [ ]:
# observamos la distribución de use_Mileage en función del precio, mediante un scatterplot.
plt.figure(figsize=(12,6))
sns.scatterplot(y=df['Price'], x=df['use_Mileage']);
plt.title('Mileage and Price relation ',fontsize=15,color='blue',fontweight='bold')

In [ ]:
##Variable "Year"
fn.count_box_plot("use_Year", df)

__Observaciones:__

- Existe una asociación positiva de 0.4 entre `Year` y el `precio` de vehículos.
- Existe una asociación negativa de 0.4 entre `Mileage` y el `precio` de los vehículos.
- Entre `year` y `Mileage` exite una fuerte asociación negativa de 0.8;  Dada esta asociación, quizás es recomendable considerar solo uno de estos atributos.

### 2.2.3 Distribución atributos localization (loc):

#### 2.2.3.1 Distribución y relación de los atributos categóricos "Loc" con el vector objetivo.

In [ ]:
#Variable loc_state
fn.count_box_plot("loc_State",df, 70000,False)

In [ ]:
# observamos la distribución de loc_City, mediante un plot
df['loc_City'].value_counts().head(30).plot(kind='barh', figsize=(6,10))
plt.title('City Distribution',fontsize=15,color='blue',fontweight='bold')

__Observaciones:__

- El precio por `State` (estado) presenta una distribución similar, esto se evidencia al observar los cuartiles y la mediana de los precios en cada uno de los estados, solo el estado `DC` presenta precios inferiores, se recomienda no considerar esta variable por el momento.

- Dada la complejidad en cuanto al número de ciudades contenidas en la base de datos, por ahora no recomendamos utilizar esta variable en el proceso de entrenamiento. 

### 2.2.4 Distribución atributos de segmentacion (sgt):

A continuación se realizaran una serie de gráficos para observar el comportamiento de los atributos asociados a la `segmentación` o tipologia de los vehículos como lo son: `Vehiculetype`, `Bodyclass`, `Make`, `Model`, `Manufacture`.

#### 2.2.4.1 Distribución y relación de los atributos categóricos "de segmentación "SGT" con el vector objetivo.

In [ ]:
#variable sgt_BodyClass
fn.count_box_plot("sgt_BodyClass",df, 100000,False)                        

In [ ]:
#Variable sgt_VehicleType
fn.count_box_plot("sgt_VehicleType", df)

In [ ]:
#Variable sgt_Manufacter
fn.count_box_plot("sgt_Manufacturer", df, 100000,False)

In [ ]:
#variable sgt_Make
fn.count_box_plot("sgt_Make", df, 500000,False)

In [ ]:
# observamos la distribución de sgt_model, mediante un plot
df['sgt_Model'].value_counts().head(30).plot(kind='barh', figsize=(6,10))
plt.title('Model Distribution',fontsize=15,color='blue',fontweight='bold')

__Observaciones:__

Los atributos `BodyClass`, `VehicleType`, `Make`, `Model` y `Manufacter` muestran variaciones de los precios por categoría. Sin embargo, existen muchas categorías para estas variables con frecuencias además muy bajas, lo que puede afectar los resultados al momento de utilizarlas para el proceso de modelación sin realizar algun tipo de segmentación previa, es por esto que se recomienda utilizar dichas variables previa a la reorganización de las categorías marginales.

### 2.2.5 Distribución atributos de fabricación (fab):

A continuación se realizarán gráficos para observar el comportamiento del atributo asociados al lugar de `fabricación`de los vehículos: `PlanCountry`.

#### 2.2.5.1 Distribución y relación de los atributos categóricos de fabricación "fab" con el vector objetivo.

In [ ]:
#Variable fab_PlantCountry
fn.count_box_plot("fab_PlantCountry",df, 100000,False)

__Observaciones:__

El atributo `PlantCountry` muestra variación de los precios entre paises, sin embargo, también se observan frecuencias de paises marginales, por lo que se recomienda utilizar esta variable para el entrenamiento de los datos, previa su recategorización.

# 3. Segmentación de variables

## 3.1 Segmentación de fab_PlantCountry

Una de las consideraciones que se debe tener presente cuando se cuenta con atributos que presentan categorías marginales, es la reagrupación o segmentación de estos, de modo tal que estas categorías minoritarias no interfieran al momento de la modelación, es por esto que para el atributo `fab_PlantCountry`, se decide segmentar en nuevas categorías el 4.3% de los datos, los cuales se encuentran distribuidos en una serie de países que presentan estas cantidades más bajas, por lo tanto se decide crear dos nuevas categorías que los agrupen.

In [ ]:
# reagrupamos el atributo fab_PlantCountry
df['fab_PlantCountry'] = df['fab_PlantCountry'].replace(
                         ['FRANCE','SPAIN', 'PORTUGAL', 'POLAND', 'NETHERLANDS', 'SERBIA', 'FINLAND',"ITALY","UNITED KINGDOM (UK)","AUSTRIA","HUNGARY","ENGLAND","BELGIUM","SWEDEN","SLOVAKIA"],'OTHERS_EUROPE').replace(
                         ['ARGENTINA','VENEZUELA', 'BRAZIL', 'UNITED STATES (USA), CANADA', 'CANADA, UNITED STATES (USA)'],'OTHERS_AMERICA').replace(
                         ['THAILAND','TURKEY', 'CHINA', 'AUSTRALIA', 'INDIA'],'OTHERS_ASIA_OCEANIA').replace(
                         ["OTHERS_AMERICA","OTHERS_ASIA_OCEANIA","SOUTH AFRICA"],"OTHER_COUNTRIES")

In [ ]:
# vusalizamos la cantidad de datos para c/u de las variables de fab_PlantCountry
df.fab_PlantCountry.value_counts()

__Observaciones:__

Las categorías que estan por debajo de los 5.613 registros, se reagrupan en dos nuevas categorias llamadas `OTHER_EUROPE` y `OTHER_COUNTRY`, pasando así de tener en un inicio treinta y dos categorias, paises de fabricación, a tan solo ocho.

## 3.2 Segmentación de sgt_BodyClass

Otra de las segmentaciones que se hace necesaria realizar es la del atributo `BodyClass`, el cual hace referencia a la  clase de vehículo que se tiene registrado, dentro de las categorías de este atributo nos encontramos con vehículos que se registran con cantidades muy marginales como lo son `Bus`, `Limousine` y `Trailer`, alcanzando estas un total del 0.015% registros, por otro lado se presentan registros bajo tipología de `Incompleto` los que alcanzan el  un total de 1.7% registros, es por esto que se decide eliminar dichas categorías, ya que son muy marginales y no presentan un aporte significativo para el modelo.

In [ ]:
# reagrupamos el atributo sgt_BodyClass
df['sgt_BodyClass'] = df['sgt_BodyClass'].replace(
                      ["Hatchback/Liftback/Notchback, Convertible/Cabriolet"],"Hatchback/Liftback/Notchback").replace(
                      ["Wagon, Sport Utility Vehicle (SUV)/Multi-Purpose Vehicle (MPV)"],"Wagon").replace(
                      ["Incomplete - Cutaway","Incomplete - Cutaway","Incomplete - Chassis Cab (Number of Cab Unknown)","Incomplete - Chassis Cab (Double Cab)","Incomplete - Stripped Chassis","Incomplete - Commercial Chassis","Incomplete - Motor Home Chassis","Incomplete - Chassis Cab (Single Cab)","Incomplete - Chassis Cab (Double Cab) "],"Incomplete").replace(
                      ["Truck"],"Sport Utility Truck (SUT)").replace(
                      ["Roadster"],"Convertible/Cabriolet").replace(
                      ["Cargo Van"],"Minivan")

In [ ]:
# reemplazamos con np.nan, aquellas variables del atributo sgt_BodyClass que tienen una frecuencia marginal 
df = df.replace({'sgt_BodyClass':{"Bus":np.NaN,"Limousine":np.NaN,'Trailer':np.NaN,"Incomplete":np.NaN}})

In [ ]:
# vusalizamos la cantidad de datos para c/u de las variables de sgt_BodyClass
df.sgt_BodyClass.value_counts()

__Observaciones:__

Se agrupan las categorías según automóviles similares, El criterio utilizado fue juicio experto, aquellas categorías que presentan una menor cantidad de registros y categorías tipificadas como `incompleto`, se decide transformarlas a NaN, dentro de las que tenemos `Bus`, `Limousine` e `Incomplete`, como resultado pasamos de tener veintiséis a un total de once clases de vehículos.

## 3.3 Segmentación de sgt_VehicleType

Al revisar el atributo `VehicleType` que hace referencia al tipo de vehiculo registrado, podemos observar que esta tiene 5 categorias de vehiculos, de las cuales tres concentran la mayor parte de los datos, el 99.6 % de estos. 

Estas categorias son: `Vehiculos de pasajeros`, `Vehiuclos Multiproposito` y `Camionetas`, es por esto que uno de los criterios de eliminación fue no considerar el 0.4% de los registros que hacen referencia a `Vehiculos incompletos` y `Buses`.

In [ ]:
# reemplazamos con np.nan, aquellas variables del atributo sgt_VehicleType que tienen una frecuencia marginal 
df = df.replace({'sgt_VehicleType':{"INCOMPLETE VEHICLE":np.NaN,"BUS":np.NaN,"TRAILER":np.NaN}})

In [ ]:
# vusalizamos los Q para c/u de las variables de sgt_VehicleType
df.sgt_VehicleType.value_counts()

__Observaciones:__

Dado que dentro del atributo `sgt_VehicleType` tenemos variables con una frecuencia muy marginal, se decide no considerarlas, por lo que se transforman a np.nan, quedandonos solo con tres categorias de vehiculo:

* De pasajeros
* Multiproposito
* Camión

Lo cual está muy relacionado con los tipos de vehiculos que se comercializan en una compra venta de vehiculos Usados.

## 3.4 Eliminación de Modelos de Vehiculos con menos de 30 observaciones

Para mejorar el rendimiento del modelo como último filtro se decidió eliminar los modelos de vehículos que presentan menos de 30 registros, esto porque se consideran como un dato marginal que no aporta mayormente al rendimiento del modelo predictivo.

In [ ]:
# Filtrar 30 modelos
modelos = df['sgt_Make'].value_counts()
df = df[df['sgt_Make'].isin(modelos[modelos > 30].index)]

## 3.5 Reagrupación de variable año

Cómo la variable año refleja la antiguedad de un vehículo al compararla con un año de referencia (2018 para la base de entrenamiento/prueba), se hace está trasnformación para generar la variable de 'edad' del vehículo.

In [ ]:
# Recodificación de variable año 
df['use_Age'] = 2018 - df['use_Year']

# 4. Preproceso

Durante esta etapa se realiza el preprocesamiento necesario de los atributos y vector objetivo para que puedan ser utilizados para entrenar y probar posteriormente en modelos de ML. Primero procedemos a liberar algo de espacio en la memoria RAM borrando las variables usadas durante la construcción del dataset, para luego proceder a la selección de atributos y el muestreo aleatorio de datos a preprocesar.

In [ ]:
# Liberar Espacio Memoria
del df_api
del df_data
del df_train
del df_test

### Selección de atributos para el entrenamiento:
Como resultado del Análisis de la distribución de los atributos y de la relación de ellos con el vector objetivo se decide seleccionar 5 atributos para la fase de entrenamiento de modelos:
0. Price (Vector Objetivo)
1. Mileage
2. BodyClass
3. VehicleType
4. Model
5. Age
6. EngineCylinders
7. DisplacementCC

In [ ]:
# Selección de variables para modelos
select_vars = ['Price', 'use_Mileage', 'use_Age', 'sample', 
               'sgt_BodyClass', 'sgt_Make', 'sgt_VehicleType',
               'feat_DisplacementCC', 'feat_EngineCylinders']

In [ ]:
# Muestra aleatoria
df_sample = df[select_vars]\
                .dropna()\
                .reset_index(drop=True)\

A través de la clase `PrepMl` se realizarán los tres preprocesos seleccionados para esta modelación: __Remove_Outliers__, __OneHot_Encoder__ y __Standard_Scaler__.

In [ ]:
# Instanciar clase para realizar preproceso
df_prep = PrepML(df_sample)

In [ ]:
# Eliminación de Outliers
df_prep.remove_outliers(['Price', 'use_Mileage', 'use_Age', 
                         'feat_DisplacementCC', 'feat_EngineCylinders'], 
                        iqr_multiplier=1.5, print_diff=True)

In [ ]:
# Realizamos OneHot Encoder a las columnas categóricas seleccionadas
df_prep.one_hot_encoder(['sgt_BodyClass', 'sgt_Make', 'sgt_VehicleType'],
                        drop_first=True)

In [ ]:
# Estandarizamos variables continuas seleccionadas
df_prep.standard_scaler(['use_Mileage', 'use_Age', 'feat_DisplacementCC', 'feat_EngineCylinders'])

In [ ]:
# Separar muestras según
X_train, y_train, X_test, y_test, X_val, y_val = df_prep.to_ml_samples('sample', 'Price')

# 5. Modelamiento 

A través de la clase `MLModel` se entrenarán y se realizará una busqueda de grilla para encontrar los mejores hiperparámetros por modelo, y posteriormente evaluar cada mejor modelo con la muestra de prueba. 

Es decir, serán cuatro modelos a entrenarse, dos paramétricos (`Ridge Regression` y `Stochastic Gradient Descent Regression`) y dos no-paramétricos (`XGBoost` y `LightGBM`, ambas implementaciones de Gradient Boosting).

Primero, se comienza con el entrenamiento de prueba de una Regresión Lineal, para corroborar que no hay errores en el preproceso:

## 5.1 Modelo Único

### 5.1.1 Ridge Regression

```python
# Establecemos parámetros a evaluar en el modelo
ridge_grid = {'alpha': [0, .001, 0.0001],
              'solver': ['sag', 'sparse_cg']}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
ridge_reg = MLModel(model=Ridge(fit_intercept=True))
# Implementación del grid search
ridge_reg.grid_search(X_train,
                      y_train,
                      param_grid=ridge_grid,
                      n_jobs=-2,
                      cv=5)
# Serialización del mejor modelo
ridge_reg.to_pickle(car_category='allcars')
```

In [ ]:
# Importamos mejor modelo
ridge_best = MLModel.from_pickle('best_models/allcars_ridge.sav')
# Métricas mejor modelo
ridge_best.train_val_metrics(X_train, y_train, X_val, y_val)

### 5.1.2 Stochastic Gradient Descent Regression (SGD)

```python
# Establecemos parámetros a evaluar en el modelo
sgd_grid = {'loss': ['squared_epsilon_insensitive', 'squared_loss'],
            'alpha': [0, 0.0001, 0.00001]
          }
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
sgd_reg = MLModel(model=SGDRegressor(penalty = 'l1',
                                     early_stopping = False,
                                     random_state=rd_seed))
# Implementación del grid search
sgd_reg.grid_search(X_train,
                    y_train,
                    param_grid=sgd_grid,
                    n_jobs=-2,
                    cv=5)
# Serialización del mejor modelo
sgd_reg.to_pickle(car_category='allcars')
```

In [ ]:
# Importamos mejor modelo
sgd_best = MLModel.from_pickle('best_models/allcars_sgdregressor.sav')
# Métricas mejor modelo
sgd_best.train_val_metrics(X_train, y_train, X_val, y_val)

### 5.1.3 LightGBM

```python
# Establecemos parámetros a evaluar en el modelo
lgb_grid = {'max_depth': [11, 12, 13], 
            'num_leaves': [125, 135, 145]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
lgb_reg = MLModel(model=LGBMRegressor(n_jobs=1,
                                      random_state=rd_seed))
# Implementación del grid search
lgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=lgb_grid,
                    n_jobs=-2,
                    cv=5)
# Serialización del mejor modelo
lgb_reg.to_pickle(car_category='allcars')
```

In [ ]:
# Importamos mejor modelo
lgb_best = MLModel.from_pickle('best_models/allcars_lgbmregressor.sav')
# Métricas mejor modelo
lgb_best.train_val_metrics(X_train, y_train, X_val, y_val)

### 5.1.4 XGBoost

```python
# Establecemos parámetros a evaluar en el modelo
xgb_grid = {'max_depth': [7, 8, 9], 
            'n_estimators': [80, 90, 100]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
xgb_reg = MLModel(model=XGBRegressor(objective ='reg:squarederror',
                                     n_jobs=1,
                                     seed=rd_seed))
# Implementación del grid search
xgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=xgb_grid,
                    n_jobs=2,
                    cv=3)
# Serialización del mejor modelo
xgb_reg.to_pickle(car_category='allcars')
```

In [ ]:
# Importamos mejor modelo
xgb_best = MLModel.from_pickle('best_models/allcars_xgbregressor.sav')
# Métricas mejor modelo
xgb_best.train_val_metrics(X_train, y_train, X_val, y_val)

__Comentarios Generales__:
    
De los resultados obtenidos podemos concluir apriori que:  
1.- Existen relaciones no lineales en la variables que se refleja en la superioridad de los modelos basados en árboles de decisión sobre los modelos lineales. Estos resultados son consistentes con los análisis  intermedios y exploratorios.  
2.- Modelando solo un modelo para calcular el precio de mercado, obtuvimos resultados que alentarían el uso de un único modelo para predecir todos los casos, aunque no se descarta la alterantiva de grupos de modelos para predecir dividendo por alguna categoría queda pendiente a evaluarse en los próximos pasos. 

## 5.2 Modelos Múltiples

Dividimos la muestra según las categorías en `VehicleType`, dado que cada las tres categorías que representa esta variable, divide los autos según la principal función del vehículo: de pasajeros, multipropósito y camiones (para carga). Esta división representaría de mejor forma la relación de los atributos en el precio del vehículo. A continuación, preparamos las muestras para entrenar y evaluar los distintos modelos por categoría de vehículo.

In [ ]:
# Creación de diccionarios con tipo de vehículos
car_dict = {'psg': 'PASSENGER CAR', 
            'mpp': 'MULTIPURPOSE PASSENGER VEHICLE (MPV)', 
            'trk': 'TRUCK '}
# Lista con tipo de muestras a crearse
sample_list = ['X_train', 'y_train', 'X_test', 'y_test', 'X_val', 'y_val']
samples = {}

for cat, car_type in car_dict.items():
    
    # Instanciar clase para realizar preproceso
    prep = PrepML(df_sample[df_sample['sgt_VehicleType'] == car_type]\
                            .reset_index(drop=True)
                            .drop(columns='sgt_VehicleType'))
    # Eliminación de Outliers
    prep.remove_outliers(['Price', 'use_Mileage', 'use_Age', 'feat_DisplacementCC', 
                             'feat_EngineCylinders'], iqr_multiplier=1.5,
                              print_diff=False)
    # Realizamos OneHot Encoder a las columnas categóricas seleccionadas
    prep.one_hot_encoder(['sgt_BodyClass', 'sgt_Make'],
                            drop_first=True)
    # Estandarizamos variables continuas seleccionadas
    prep.standard_scaler(['use_Mileage', 'use_Age', 'feat_DisplacementCC', 
                             'feat_EngineCylinders'])
    # Separar muestras
    samples[cat] = {i: j for i, j in zip(sample_list, 
                                         prep.to_ml_samples('sample', 'Price'))}
    # Agregamos los transformadores
    samples[cat].update({'transformers': prep.transformers})
    # Agregamos muestra para entrenar el objeto ColumnTransformer para serializar los modelos
    samples[cat].update({'df_ct': prep.df_ct})

Definimos los modelos y grillas de hiperparámetros a evaluarse en las 3 categorías de vehículos:

In [ ]:
# Lista con modelos
ridge_model = Ridge(fit_intercept=True)
sgd_model = SGDRegressor(penalty = 'l1',
                       early_stopping = False,
                       random_state=rd_seed)
lgb_model = LGBMRegressor(n_jobs=1,
                        random_state=rd_seed)
xgb_model = XGBRegressor(objective ='reg:squarederror',
                       n_jobs=1,
                       seed=rd_seed)

model_list = [ridge_model, sgd_model, lgb_model, xgb_model]

# Lista con grilla de hiperparámetros
ridge_grid = {'alpha': [0, .001, 0.0001],
              'solver': ['sag', 'sparse_cg']
             }
sgd_grid = {'loss': ['squared_epsilon_insensitive', 'squared_loss'],
            'alpha': [0, 0.0001, 0.00001]
           }
lgb_grid = {'max_depth': [11, 12, 13], 
           'num_leaves': [110, 120, 130]
           }
xgb_grid = {'max_depth': [7, 8, 9], 
            'n_estimators': [80, 90, 100]
           }

grid_list= [ridge_grid, sgd_grid, lgb_grid, xgb_grid]

Ahora pasamos a entrenar los modelos por categoría de `VehicleType`

### 5.2.1 Modelos Passenger Car

In [ ]:
# Definimos la categoría de vehículo a modelar 
category = 'psg'

```python
# Entrenamos, ajustamos hiperparámetros y serializamos modelos
# (muestra de lo que hace la función train_mlmodels)
for model, grid in zip(model_list, grid_list):
    
    print(f'{category}_{model.__class__.__name__.lower()}')
    # Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
    model_reg = MLModel(model=model)
    # Implementación del grid search
    model_reg.grid_search(samples[category]['X_train'],
                          samples[category]['y_train'],
                          param_grid=grid,
                          n_jobs=-2,
                          cv=5)
    # Serialización del mejor modelo
    model_reg.to_pickle(car_category=category)
    print('\n')
```

In [ ]:
# Importamos y evaluamos los modelos serializados 
# (muestra de lo que hace la función 'metrics_pickled_mlmodels')
pickle_files = [f'{category}_{model.__class__.__name__.lower()}.sav' for model in model_list]

for pickle_model in pickle_files:
    # Importamos mejor modelo
    best_model = MLModel.from_pickle(f'best_models/{pickle_model}')
    # Métricas mejor modelo
    print(pickle_model[:-4])
    print(best_model.train_val_metrics(samples[category]['X_train'], 
                                      samples[category]['y_train'], 
                                      samples[category]['X_val'], 
                                      samples[category]['y_val']))
    print('\n')

### 5.2.2 Modelos Multipurpose Passenger Vehicle

In [ ]:
# Definimos la categoría de vehículo a modelar
category = 'mpp'

```python
# Entrenamos, ajustamos hiperparámetros y serializamos modelos
fn.train_mlmodels(model_list, grid_list, samples, category)
```

In [ ]:
# Importamos y evaluamos los modelos serializados
fn.metrics_pickled_mlmodels(model_list, samples, category)

### 5.2.3 Modelos Multipurpose Passenger Vehicle

In [ ]:
# Definimos la categoría de vehículo a modelar
category = 'trk'

```python
# Entrenamos, ajustamos hiperparámetros y serializamos modelos
fn.train_mlmodels(model_list, grid_list, samples, category)
```

In [ ]:
# Importamos y evaluamos los modelos serializados
fn.metrics_pickled_mlmodels(model_list, samples, category)

Conclusión entrenamiento y evaluación modelos por categorías:
* Se observa que al igual con el modelo único, los modelos no paramétricos superan en resultados a los modelos paramétricos, siendo `lightgbm`  marginalmente el mejor entre los 4 modelos (evaluando métrica de MAE en validación y su diferencia con respecto a la muestra de entrenamiento).
* Se observa en términos generales resultados similares entre las muestras, con un mínimo r2 en muestra de prueba .82 en la categoría `Passenger` y un máximo de .89 en la categoría `Multipurpose Passenger`.

Para poder decidir con que tipo de modelo, único o múltiple, se decide utilizar, se evaluará su rendimiento en la muestra de validación.

## 5.3 Evaluación con muestra de Prueba (Hold-out sample)

In [ ]:
# Evaluamos nuestro mejor modelo en la muestra de prueba
{key: value for key, value in lgb_best.metrics(X_test, y_test).items() if key != 'r2'}

In [ ]:
# Importamos los mejores modelos
psg_best = MLModel.from_pickle('best_models/psg_lgbmregressor.sav')
mpp_best = MLModel.from_pickle('best_models/mpp_lgbmregressor.sav')
trk_best = MLModel.from_pickle('best_models/trk_lgbmregressor.sav')
# Diccionario con mejores modelos
car_models = {'psg': psg_best, 'mpp': mpp_best, 'trk': trk_best}

for category, model in car_models.items():
    
    metrics = model.metrics(samples[category]['X_test'], samples[category]['y_test'])
    # Evaluamos en la muestra de validación
    print(f'{category}_{model.best_model.__class__.__name__.lower()}')
    print({key: value for key, value in metrics.items() if key != 'r2'})
    print('\n')

De los resultados en las métricas de los modelos en la muestra de prueba, se pueden observar resultados similares a los obtenidos en la muestra de entrenamiento y validación. En el caso del modelo único, el MAE es practiamente el mismo, manteniendose la distancia con respecto al RSME.

Por su parte, la modelación múltiple presenta resultados positivos en las catogorías de 'psg', 'mpp' y 'trk', siendo muy acotadas las diferencias en el MAE y r2 de la muestra de validación y de prueba, siendo la diferencia con RSME igualmente similar. 

Cómo se observa en los resultados de las métricas, la diferencia entre el MAE y el RSME es una constante tanto para las muestra de prueba y validación, especialmente para el caso de 'trk'. Esta situación daría indicios de que el posiblemente modelo está subestimando el precio de vehículos, especialmente el de los segmentos más caros. Para corroborar esta hipótesis, observaremos la distribución de los precios en las muestras de prueba.

In [ ]:
# Grafico de boxplot para observar la distribución de los precios por muestra
box_list = []
car_types = ['psg', 'mpp', 'trk']
for category in car_types:
    box_list += [samples[category]['y_test']]

plt.boxplot(box_list, labels=car_types, vert=False)
plt.show()

En conclusión:
* Se observa que las ditribuciones de las muestras de validación siguen presentando un número de casos posibles de denominar como 'outliers', sinedo especialmente drámatico el caso de la muestra de 'psg'.
* Por lo tanto, sea modelo único o múltiple, se observan las limitaciones de la modelación, en los casos de vehículos de alta gama, cuyos precios superarían ciertos rangos, dependiendo del tipo de vehículo.
* A pesar de esta limitación, los rendimientos por separados muestran un marginalmente un mejor desempeño en datos no observados por el modelo al entrenarse, por lo que se prefería continuar con los modelos múltiples por tipo de vehículo para desarrollar la solución.

## 5.4 Análisis de Distribución de Errores

In [ ]:
# generación de lista con bases de Prueba con atributos.
X_test_list=[X_test,
            samples["psg"]["X_test"],
            samples["mpp"]["X_test"],
            samples["trk"]["X_test"]]
# generación de lista con bases de Prueba con vector objetivo.
y_test_list=[y_test,
            samples["psg"]["y_test"],
            samples["mpp"]["y_test"],
            samples["trk"]["y_test"]]
# generación de lista con modelos.
model_list=[lgb_best.best_model, psg_best.best_model,
            mpp_best.best_model,trk_best.best_model]

In [ ]:
# QQ Plot
fn.qq_plot(model_list,X_test_list, y_test_list)

__Comentarios__:
De manera adicional a las métricas de RMSE y MAE, proponemos observar la conducta de los modelos con un QQ plot. Este gráfico comparara la distribución de los errores (residuos) con la distribución normal. Los gráficos muestran que los errores de los modelos tienen buen comportamiento entre los cuartiles -4 y 2. La situación cambia con los cuartiles 2 y 4. Nuestros modelos fallan en mayor medida para autos de valores altos.

## 5.5 Feature importaces

In [ ]:
# feature importance modelo de pasageros psg
plt.rcParams['figure.figsize'] = (3.3, 5)
fn.grafico_importancia(model_list[1],X_test_list[1].columns)
plt.xlabel("Importancia relativa");
plt.ylabel('Atributos');

In [ ]:
# feature importance modelo de multipropósiro (mpp)
plt.rcParams['figure.figsize'] = (5, 5)
fn.grafico_importancia(model_list[2],X_test_list[2].columns)
plt.xlabel("Importancia relativa");
plt.ylabel('Atributos');

In [ ]:
# feature importance modelo de truk (trk)
plt.rcParams['figure.figsize'] = (5.5, 5)
fn.grafico_importancia(model_list[3],X_test_list[3].columns)
plt.xlabel("Importancia relativa");
plt.ylabel('Atributos');

__Comentarios__:
Para los tres tipos de vehículos los atributos más importantes son Mileage (kilométrajes), Displacement (cilindrada) y use_age (antiguedad del vehículo).

## 5.6 Pipeline y Serialización

In [ ]:
# Serializamos modelo único
# Generamos el objeto pipeline con nuestro mejor modelo y transformadores entrenados
pipeline = lgb_best.to_pipeline(df_prep.transformers, 
                                df_prep.df_ct.drop(columns=['Price', 'sample']))
# Serializamos el pipeline
pickle.dump(pipeline, open('best_models/pipeline_allcars.sav', 'wb'))

In [ ]:
# Serializamos modelo múltiple
for category, model in car_models.items():
    
    # Generamos el objeto pipeline con nuestro mejor modelo y transformadores entrenados
    pipe = model.to_pipeline(samples[category]['transformers'], 
                             samples[category]['df_ct'].drop(columns=['Price', 'sample']))
    # Serializamos el pipeline
    pickle.dump(pipe, open(f'best_models/pipeline_{category}.sav', 'wb'))

# 6. Política de Recomendación

Habiendo definido un modelo que posbilite la estimación de un __precio de mercado__, para poder ofrecer una recomendación a través de la aplicación desarrollada debemos ofrecer una evaluación de posibles alternativas de compras. Actualmente podemos calcular el potencial margen de una oferta a través de la siguiente formula:
 
$$ m_{usd} = P_{estimado} - P_{ofertado}$$  
Siendo el $P_{estimado}$ el precio estimado por nuestro modelo y $P_{ofertado}$ el precio ofertado para poder adquirir el vehículo.  
Por otro lado, podemos establecer la misma métrica en términos porcentuales:

$$ m_{pct} = \frac{P_{estimado} - P_{ofertado}} {P_{estimado}}$$ 

Lo cual nos permite establecer una polítca de compra en base a términos porcentuales de margene requerido para considerar como una buena compra. Cómo el mercado automotriz es bien competitivo, especialmente en Estados Unidos, podemos definir que una compra con al menos un 8% de margen sea considerado como una compra recomendada, lo que podemos expresar con la siguiente formula:  

$$ m_{pct} - 0.08 > 0 => {Recomendado-comprar}$$

Pero cómo cualquier modelo de estimación de precios, nuestro modelo no es infalible. Cómo se mostró en la sección anterior, dependiendo del vehículo, este puede fallar en mayor o menor envergadura. Con la información utilizada para entrenar el modelo, podemos calcular el error en términos porcentuales para cada observación de esta muestra. De este cálculo podemos, generar la media de este error por modelo de vehículo, que es una variable que engloba muchos atributos relevantes del vehículo (ej: marca, tipo chasís, etc.) y por lo tanto de importancia para el precio, y por lo tanto generar un proxy estimativo del error que generaría el modelo ante una nueva observación. Dicho error histórico se puede representar de la siguiente forma:  

Error Histórico Promedio para Modelo j:   

$$ e_j = \frac {1}{n} \sum_{i=1}^{i\in j} \frac{P_{hist-estimado}-P_{hist-ofrecido}} {P_{hist-estimado}}$$


In [ ]:
# Creamos la base para calcular los errores 
df_aux = df.loc[:, select_vars + ['sgt_Model']].dropna().reset_index(drop=True)
df_aux['Marca'] = df_aux['sgt_Make']
aux_dict = {value: key for key, value in car_dict.items()}
df_aux['ml_model'] = df_aux['sgt_VehicleType'].map(aux_dict)
df_aux = df_aux.drop(columns='sgt_VehicleType').rename(columns={'sgt_Model': 'Modelo'})
# Instanciamos la clase para preprocesamiento
prep_aux = PrepML(df_aux)
# Eliminación de Outliers
prep_aux.remove_outliers(['Price', 'use_Mileage', 'use_Age', 'feat_DisplacementCC', 
                         'feat_EngineCylinders'], iqr_multiplier=1.5,
                          print_diff=False)
# Realizamos OneHot Encoder a las columnas categóricas seleccionadas
prep_aux.one_hot_encoder(['sgt_BodyClass', 'sgt_Make'],
                        drop_first=True)
# Estandarizamos variables continuas seleccionadas
prep_aux.standard_scaler(['use_Mileage', 'use_Age', 'feat_DisplacementCC', 
                         'feat_EngineCylinders'])
# Eliminamos la variable que separaba las muestras
df_aux = prep_aux.df.drop(columns='sample')
# Cambio de tipo de dato para 'ml_model'
ml_type = CategoricalDtype(categories=['psg', 'mpp', 'trk'], ordered=True)
df_aux = df_aux.astype({'ml_model': ml_type}).sort_values(by='ml_model')

In [ ]:
# Realizamos las predicciones por categoría de vehículo
y_hat = []
for category, model in car_models.items(): 
    y_hat += list(model.best_model.predict(df_aux[df_aux['ml_model'] == category]\
                                               .loc[:, samples[category]['X_train'].columns]
                                          ))
# Redondeamos y pasamos a integer las predicciones
y_hat = [int(round(i, 0)) for i in y_hat] 

In [ ]:
# Creamos las variables 'y_hat' y 'error'
df_aux['y_hat'] = y_hat
df_aux['error'] = (df_aux['y_hat'] - df_aux['Price'])/df_aux['y_hat']
df_aux['error'] = df_aux['error'].map(lambda x: round(x, 2))

In [ ]:
# Error Promedio por Marca/Modelo
error_df = df_aux.loc[:, ['Marca', 'Modelo', 'Price', 'y_hat', 'error']]\
            .groupby(by=['Marca', 'Modelo'])\
            .agg({'Price': ['mean', 'count'], 'y_hat': 'mean', 'error': 'mean'})\
            .round(2)
error_df

Como se puede observar del cálculo de error por modelo, este puede diferir considerablemente y en distintas direcciones dependiendo del modelo del vehículo, subestimando o sobrestimando el precio de manera importante en ciertos casos. Es por ello que proponemos usar esta error histórico para ajustar el margen generado por el precio ofrecido y el precio estimado, de la siguiente forma:

Margen ajustado para oferta i de modelo j de vehículo:  
$$ ma_i = m_i - e_j $$

Pero si observamos más a fondo el error histórico, nos encontramos con que a pesar de la corrección, la estimación puede ser muy poco acertada, por lo que es necesario generar matices a esta correción:

In [ ]:
# Distribución del error histórico
ax = sns.boxplot(error_df['error'])
plt.title('Boxplot errores históricos por modelo de vehículo')
plt.show()

In [ ]:
# Cuartiles y rango intercuartil de boxplot
q1 = error_df['error'].quantile(0.25).values[0]
q3 = error_df['error'].quantile(0.75).values[0]
iqr = q3 - q1
print(f'Primer Cuartil: {q1}\nTercer Cuartil: {q3}')
print(f'Bigote Inferior:{round(q1 - iqr*1.5, 2)}\nBigote Superior:{round(q3 + iqr*1.5, 2)}')


Por lo que para evitar malas recomendaciones, se evitará entregar una recomendación cuyos modelos de vehículos tengan precio de mercado con errores históricos superiores al $0.6$ o inferiores a $-0.72$, dado que aún con la correción, la estimación puede ser muy poco fíable. Bajo el mismo concepto, la recomendación se hará con reservas si, habiendose cumplido las condiciones de margen ajustado mayor a 8%, el error histórico promedio del modelo no se encuentra entre el primer y tercer quintil de la distribución de los errores históricos. Finalmente, a continuación se resume como la política de recomendación se establecería:

Para un oferta $i$ de modelo de vehículo $j$ con un margen mínimo desado de 8%:

`if` $(e_j <-0.72$ `or` $ e_j > 0.6)$ => `No hay Recomendación`   
`elif` $(m_i - e_j < 0.08 ) =>$ `No Recomendado para comprar`  
`elif` $(-0.23 <= e_j <= 0.1)$ `&` $(m_i - e_j >= 0.08) =>$ `Recomendado comprar con seguridad`  
`elif` $(e_j <-0.23$ `or` $ e_j > 0.1$) `&` ($m_i - e_j >= 0.08) =>$ `Recomendado comprar con reservas`  


# 7. Conclusión

Para predecir un Precio de Mercado de un vehículo usado a través de sus atributos propios o de uso, conluímos que utilizar una serie de modelos por tipo de vehículo (de pasajeros, multiproposito o camión) genera mejores resultados en la muestra de prueba (r2 promedio .85) y de validación (r2 promedio de .7 sin remover outliers).

Como lo son vehículos de pasajeros, multiproposito y camionetas, esto porque sus carasterísticas técnicas y precios son bastante diferentes y nos obligan a buscar resultados mas precisos gracias a esta segmentación, respecto a la precisión al momento de predecir vehículos de un mayor valor, o también denominados de alta gamma, nuestro modelo tiende a aumentar el error en estos casos, esto porque la mayor parte de los precios en la data de entrenamiento se situan entre los 10.000 y 37.000 USD, por lo que para los casos que están por sobre el promedio se genera lo que conocemos como underfitting y nuestro modelo no logra predecir con exactitud el precio requerido.

Si bien una de las alternativas posibles y recomendadas al igual que se hizo con las tipologias de vehículos, sería generar nuevos modelos para gammas bajas y gammas altas de vehículos, en esta ocación esa alternativa no es viable dado que contamos con un numero de datos para los casos de vehículos caros que no logra la cantidad suficiente de muestra, que nos permita realizar un entrenamiento del modelo para esta tipología. Por ende para lograr esa solución se deben buscar nuevos datos que se asemejen a los de alta gamma.

Cabe destacar que la función principal del modelo a traves de la aplicación web es generar una recomendación en función del precio estimado para las características técnicas y de uso de cada vehículo, lo cual se cumple y logra de manera bastante eficiente, pero es importante tener en consideración que existen factores estéticos, técnicos, de demanda del modelo y rotación de este, y también relacionados al cuidado del vehículo (considerando que son usados), que se deben tener presente al momento de definir si un determinado vehículo es o no una buen negocio para la automotora.